In [1]:
import kfp
from kfp.dsl import component, pipeline

In [2]:
@component(
    base_image='python:3.9',
    packages_to_install=["git+https://github.com/kubeflow/trainer.git@master#subdirectory=sdk"],
)
def get_job_logs_n_cleanup(job_id: str):
    from kubeflow.trainer import CustomTrainer, TrainerClient

    for s in TrainerClient().get_job(name=job_id).steps:
        print(f"Step: {s.name}, Status: {s.status}, Devices: {s.device} x {s.device_count}")
    
    logs = TrainerClient().get_job_logs(name=job_id)
    
    print(logs["node-0"])
    
    TrainerClient().delete_job(name=job_id)

In [5]:
@pipeline(name='get-job-logs-n-cleanup-pipeline')
def get_job_logs_n_cleanup_pipeline():
    step1 = get_job_logs_n_cleanup(job_id="kcd9605c904a")

client = kfp.Client()
kfp.compiler.Compiler().compile(get_job_logs_n_cleanup_pipeline, 'get_job_logs_n_cleanup_pipeline.yaml')
run = client.create_run_from_pipeline_func(get_job_logs_n_cleanup_pipeline, arguments={}, enable_caching=False)

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(
